# Rock, Paper, Scissors

**Import Custom Scripts [Local Notebook]**

Use this import if you are running the notebook locally from the root directory of the github repository

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_addons as tfa
import matplotlib.pyplot as plt

from pathlib import Path

from lib.image_transforms import stateless_random_rotate

**Import Custom Scripts [Google Colab]**

To use the custom scripts in Google Colab, clone the git repository to your local directory and use an absolute import statement.

Check out `./learning_tf/lib` for the custom functions used in this notebook

In [ ]:
# ! pip install tensorflow-addons
# ! git clone https://github.com/junpeng-jp/learning_tensorflow ./learning_tensorflow

# import tensorflow as tf
# import tensorflow_datasets as tfds
# import tensorflow_addons as tfa
# import matplotlib.pyplot as plt

# from pathlib import Path

# from learning_tensorflow.lib.image_transforms import stateless_random_rotate

## Downloading Dataset

Tensorflow datasets is a library that provides user-friendly APIs to download many datasets available online. A convenient Split API is included that will split the downloaded files into `tf.record` files.

In [ ]:
builder = tfds.builder(
    'rock_paper_scissors',
    data_dir = "./data")

builder.download_and_prepare()

train, test = builder.as_dataset(
    split = ['train', 'test'],
    shuffle_files = True)

In [ ]:
builder.info

In [ ]:
label_encoder = builder.info.features['label']

In [ ]:
seed = 23916481831
tf.random.set_seed(seed)

## Classifying CG Hands

The dataset provided uses a images of hands that were generated using computer graphics software. Most of the hand poses are slight variations in angles of a base CG hand and can potentially introduce some biases into the convolution model that we will be using.

In [ ]:
fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(8,8))
fig.tight_layout()

for ax, data in zip(axes.ravel(), train.take(16)):
    ax.imshow(data['image'].numpy())
    ax.set_title("{}".format(label_encoder.int2str(data['label'])))
    ax.axis("off")

In [ ]:
IMG_SIZE = (300, 300)
DELTA_ROTATE = 45
DELTA_BRIGHTNESS = 0.2
DELTA_HUE = 0.5
DELTA_SATURATION = (0.8, 1.2)
rng = tf.random.Generator.from_seed(seed)

def standardize_img(data):
    image, label = data['image'], data['label']
    image = tf.cast(image, tf.float64) / 255.
    image = tf.image.resize(image, IMG_SIZE) 
    label = tf.one_hot(label, depth = 3)
    
    return image, label

def augmentation(image, label):
    rng_seed = rng.make_seeds(1)[:, 0]
    
    image = tf.image.stateless_random_brightness(image, DELTA_BRIGHTNESS, seed = rng_seed)
    image = tf.image.stateless_random_flip_left_right(image, seed = rng_seed)
    image = tf.image.stateless_random_flip_up_down(image, seed = rng_seed)
    image = stateless_random_rotate(image, DELTA_ROTATE, seed = rng_seed, fill_value=0)
    image = tf.image.stateless_random_saturation(image, *DELTA_SATURATION, seed = rng_seed)
    image = tf.clip_by_value(image, 0, 1)
    
    return image, label

In [ ]:
BATCH_SIZE = 32

trainloader = (
    train
    .shuffle(1000)
    .map(standardize_img, num_parallel_calls = tf.data.experimental.AUTOTUNE)
    .batch(BATCH_SIZE)
    .map(augmentation, num_parallel_calls = tf.data.experimental.AUTOTUNE)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

testloader = (
    test
    .map(standardize_img, num_parallel_calls = tf.data.experimental.AUTOTUNE)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

In [ ]:
fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(8,8))
fig.tight_layout()

for img, label in trainloader.take(1):
    label = tf.argmax(label, axis = 1)
    for ax, i in zip(axes.ravel(), range(16)):
        ax.imshow(img[i].numpy())
        ax.set_title("{}".format(label_encoder.int2str(label[i])))
        ax.axis("off")

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(*IMG_SIZE, 3)))

model.add(tf.keras.layers.Conv2D(64, (3,3), activation='relu', padding='same', kernel_initializer='he_normal'))
model.add(tf.keras.layers.MaxPool2D())
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Conv2D(64, (3,3), activation='relu', padding='same', kernel_initializer='he_normal'))
model.add(tf.keras.layers.MaxPool2D())
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Conv2D(128, (3,3), activation='relu', padding='same', kernel_initializer='he_normal'))
model.add(tf.keras.layers.MaxPool2D())
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Conv2D(128, (3,3), activation='relu', padding='same', kernel_initializer='he_normal'))
model.add(tf.keras.layers.MaxPool2D())
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dense(3, activation='softmax'))

In [ ]:
optimizer = tfa.optimizers.AdamW(weight_decay=1e-6, learning_rate=0.0001)

model.compile(
    optimizer = optimizer, 
    loss = 'categorical_crossentropy',
    metrics=['accuracy'])

In [ ]:
model.fit(
    trainloader,
    validation_data = testloader,
    epochs = 30)